(batch_inference_overview)=
# Batch inference

Batch inference or offline inference addresses the need to run machine learning model on large datasets. It is the process of generating outputs on a batch of observations.

With batch inference, the batch runs are typically generated during some recurring schedule (e.g., hourly, or daily). These inferences are then stored in a database or a file and can be made available to developers or end users. With batch inference, the goal is usually tied to time constraints and the service-level agreement (SLA) of the job. Conversely, in real time serving, the goal is usually to optimize the number of transactions per second that the model can process. An online application displays a result to the user.

Batch inference can sometimes take advantage of big data technologies, such as Spark, to generate predictions. Big data technologies allow data scientists and machine learning engineers to take advantage of scalable compute resources to generate many predictions simultaneously. To gain a better understanding about the batch inference usage and the function parameters, see the [Batch Inference page on the Function Hub](https://www.mlrun.org/hub/functions/master/batch_inference_v2/).

## Test your model

To evaluate batch model prior to deployment, you should use the `evaluate` handler of the `auto_trainer` function.

This is typically done during model development. For more information refer to the {ref}`auto_trainer_evaluate` handler documentation. For example:
``` python
import mlrun

# Set the base project name
project_name_base = 'batch-inference'

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name_base, context="./", user_project=True)

auto_trainer = project.set_function(mlrun.import_function("hub://auto_trainer"))

evaluate_run = project.run_function(
    auto_trainer,
    handler="evaluate",
    inputs={"dataset": train_run.outputs['test_set']},
    params={
        "model": train_run.outputs['model'],
        "label_columns": "labels",
    },
)
```

## Deploy your model

Batch inference is implemented in MLRun by running the function with an input dataset. With MLRun you can easily create any custom logic in a function, including loading a model and calling it.

The Function Hub [batch inference function](https://github.com/mlrun/functions/tree/master/batch_inference_v2) is used for running the models in batch as well as performing drift analysis. The function supports the following frameworks:

- Scikit-learn
- XGBoost
- LightGBM  
- Tensorflow/Keras
- PyTorch
- ONNX

Internally the function uses MLRun's out-of-the-box capability to load run a model via the {py:class}`mlrun.frameworks.auto_mlrun.auto_mlrun.AutoMLRun` class.

### Basic example
The simplest example to run the function is as follows:

#### Create project

Import MLRun and create a project:

In [1]:
import mlrun

project = mlrun.get_or_create_project(
    "batch-inference", context="./", user_project=True
)
batch_inference = mlrun.import_function("hub://batch_inference_v2")

#### Get the model

Get the model. The model is a [decision tree classifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) from scikit-learn. Note that if you previously trained your model using MLRun, you can reference the model artifact produced during that training process.

In [2]:
import sys

suffix = (
    mlrun.__version__.split("-")[0].replace(".", "_")
    if sys.version_info[1] > 7
    else "3.7"
)

model_path = mlrun.get_sample_path(f"models/batch-predict/model-{suffix}.pkl")

model_artifact = project.log_model(
    key="model", model_file=model_path, framework="sklearn"
)

#### Get the data

Get the dataset to perform the inference. The dataset is in `parquet` format.

In [3]:
prediction_set_path = mlrun.get_sample_path("data/batch-predict/prediction_set.parquet")

#### Run the batch inference function    

Run the inference. In the first example we will not perform any drift analysis

In [4]:
batch_run = project.run_function(
    batch_inference,
    inputs={"dataset": prediction_set_path, "model_path": model_artifact.uri},
)

#### Function output

The output of the function is an artifact called `prediction`:

In [5]:
batch_run.artifact("prediction").as_df().head()

#### View the results in the UI

The output is saved as a parquet file under the project artifact path. In the UI you can go to the `batch-inference-infer` job --> artifact tab to view the details.

![batch prediction results](../_static/images/batch_inference_prediction_artifact.png)

### Scheduling a batch run

To schedule a run, you can set the schedule parameter of the run method. The scheduling is done by using a cron format.

You can also schedule runs from the dashboard. On the Projects > Jobs and Workflows page, you can create a new job using the New Job wizard. At the end of the wizard, you can set the job scheduling. In the following example, the job is set to run every 30 minutes.


In [6]:
batch_run = project.run_function(
    batch_inference,
    inputs={"dataset": prediction_set_path, "model_path": model_artifact.uri},
    schedule="*/30 * * * *",
)

### Drift analysis

By default, if a model has a sample set statistics, `batch_inference` performs drift analysis and produces a data drift table artifact, as well as numerical drift metrics. In addition, this function either creates or updates an existing [model endpoint](../monitoring/model-monitoring-deployment.html#:~:text=Model%20Endpoint%20%E2%80%94%20A%20combination%20of%20a%20deployed%20Nuclio%20function%20and%20the%20models%20themselves.%20One%20function%20can%20run%20multiple%20endpoints%3B%20however%2C%20statistics%20are%20saved%20per%20endpoint.) record (depends on the provided `endpoint_id`). 

In addition, you can define `"trigger_monitoring_job": True` to trigger the drift job analysis immediately. 

To provide sample set statistics for the model you can either:

1. Train the model using MLRun. This allows you to create the sample set during training.
2. Log an external model using `project.log_model` method and provide the training set in the `training_set` parameter.
3. Provide the set explicitly when calling the `batch_inference` function via the `model_endpoint_sample_set` input.

In the example below, we will provide the training set as the sample set


In [10]:
training_set_path = mlrun.get_sample_path("data/batch-predict/training_set.parquet")

batch_run = project.run_function(
    batch_inference,
    inputs={
        "dataset": prediction_set_path,
        "model_endpoint_sample_set": training_set_path,
        "model_path": model_artifact.uri,
    },
    params={
        "label_columns": "label",
        "perform_drift_analysis": True,
        "trigger_monitoring_job": True,
    },
)

In this case, instead of just prediction, you get drift analysis. If no label column was provided, the job function tries to retrieve the label columns from the logged model artifact. If also not defined in the model, the label columns are generated with the following format `predicted_label_{i}` where `i` is an incremental number. 

The drift table plot that compares the drift between the training data and prediction data per feature:

In [11]:
batch_run.artifact("drift_table_plot").show()

![batch inference drift table plot](../tutorials/_static/images/drift_table_plot.png)

You also get a numerical drift metric and boolean flag denoting whether or not data drift is detected:

In [12]:
print(batch_run.status.results)

In [13]:
# Data/concept drift per feature (use batch_run.artifact("features_drift_results").get() to obtain the raw data)
batch_run.artifact("features_drift_results").show()